In [28]:
import pandas as pd
import sklearn.preprocessing as skpre
data=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\전체자료_(강우량추가).xlsx")


In [29]:
# 침수여부
data["침수여부"]=0
data.loc[data["침수횟수"]>=1,"침수여부"]=1
data["침수여부"].value_counts()
data['구단위시우량']=data['동단위우량']/data["동면적(k㎡)"]
# 면적당 "제방","수문","암거"
data["면적당제방"]=data["제방"]/data["동면적(k㎡)"]
data["면적당수문"]=data["수문"]/data["동면적(k㎡)"]
data["면적당암거"]=data["암거"]/data["동면적(k㎡)"]
data['면적당노후건물개수']=data["노후건물개수"]/data["동면적(k㎡)"]
data["면적당지하건물"]=data["지하건물"]/data["동면적(k㎡)"]

In [30]:
data.columns

Index(['Unnamed: 0', 'GU_NAME', 'ADM_NM', '동면적(k㎡)', '인구수', '인구밀도(명/km²)',
       '동단위우량', '동단위시우량', '침수횟수', '경사도', '양호비율', '도로_건물_면적_비율', '녹지_면적_비율',
       '제방', '수문', '암거', '구난시설', '노후건물개수', '노후_건물_밀도(동/km²)', '행정동_코드',
       '월평균소득', '면적당_소비액(백만원/km²)', '분기평균총지출', '고령자비율', '장애인비율', '행정동코드', '면적',
       '지하건물', '지하건물_밀도(동/km²)', '빗물펌프장까지의거리', '평균고도', '하천까지의거리', '일일최대강수량',
       '집중호우_발생횟수', '하천면적', '하천면적/면적', '침수여부', '구단위시우량', '면적당제방', '면적당수문',
       '면적당암거', '면적당노후건물개수', '면적당지하건물'],
      dtype='object')

In [31]:
usenot_column=['Unnamed: 0', 'GU_NAME', 'ADM_NM', '동면적(k㎡)',
                '인구수','동단위우량', '동단위시우량', '침수횟수',
                '제방', '수문', '암거', '구난시설', '노후건물개수', '노후_건물_밀도(동/km²)', '행정동_코드',
                '면적당_소비액(백만원/km²)','고령자비율', '장애인비율', '행정동코드', '면적',
       '지하건물', '지하건물_밀도(동/km²)', '하천면적','침수여부','월평균소득']
numeric_column=data.columns.difference(usenot_column)

In [32]:
sca_df=data.copy()
minmax=skpre.MinMaxScaler()
minmax.fit(sca_df[numeric_column])
sca_df[numeric_column]=minmax.transform(sca_df[numeric_column])

In [33]:
# from scipy.stats import ttest_ind

# # Assuming you have a DataFrame with a '침수여부' column and other numeric columns for the variables you want to compare
# # Replace '침수여부' with the actual column name in your DataFrame

# categorical_column = "침수여부"

# # Iterate through numeric columns
# for numeric_column in data.select_dtypes(include="number").columns:
#     if numeric_column != categorical_column:
#         group1 = sca_df[sca_df[categorical_column] == 1][numeric_column]
#         group2 = sca_df[sca_df[categorical_column] == 0][numeric_column]

#         # Perform t-test
#         t_statistic, p_value = ttest_ind(group1, group2, equal_var=False)

#         # Print the results for each column
#         print(f"T-test for {numeric_column}:")
#         print("  T-statistic:", t_statistic)
#         print("  P-value:", p_value)

#         # Check for statistical significance
#         alpha = 0.1
#         if p_value < alpha:
#             print(
#                 "  Reject the null hypothesis. There is a significant difference between groups."
#             )
#         else:
#             print(
#                 "  Fail to reject the null hypothesis. There is no significant difference between groups."
#             )

#         print("\n")


In [34]:
categorical_column = "침수여부"
from scipy.stats import ttest_ind


result=[]
for x in numeric_column:
    group1 = sca_df[sca_df[categorical_column] == 1][x]
    group2 = sca_df[sca_df[categorical_column] == 0][x]

    # Perform t-test
    t_statistic, p_value = ttest_ind(group1, group2, equal_var=False)

    # Print the results for each column
    sub_result={"Feature":x,"T-statistic":t_statistic,"P-value:":p_value}

    # Check for statistical significance
    alpha = 0.1
    if p_value < alpha:
        #sub_result["comment"]="Reject the null hypothesis. There is a significant difference between groups."
        sub_result["검정결과"]="유의미함"
        result.append(sub_result)
    else:
        #sub_result["comment"]="Fail to reject the null hypothesis. There is no significant difference between groups."
        sub_result["검정결과"]="유의미하지않음"
        result.append(sub_result)


In [35]:
result_frame=pd.DataFrame(result)

In [36]:
result_frame.sort_values("검정결과",ascending=False)

,Feature,T-statistic,P-value:,검정결과
2,녹지_면적_비율,-2.220030,0.027300,유의미함
3,도로_건물_면적_비율,2.657490,0.008283,유의미함
4,면적당노후건물개수,3.074329,0.002297,유의미함
16,하천까지의거리,-2.778680,0.005950,유의미함
8,면적당지하건물,3.841342,0.000147,유의미함
1,구단위시우량,3.463020,0.000621,유의미함
13,일일최대강수량,2.477491,0.013911,유의미함
0,경사도,0.230574,0.817815,유의미하지않음
12,인구밀도(명/km²),0.113898,0.909404,유의미하지않음
15,평균고도,1.426690,0.154728,유의미하지않음


In [37]:
useful_column=result_frame.loc[result_frame["검정결과"]=="유의미함",'Feature']

In [40]:
print(useful_column.values)

['구단위시우량' '녹지_면적_비율' '도로_건물_면적_비율' '면적당노후건물개수' '면적당지하건물' '일일최대강수량'
 '하천까지의거리']


: 